In [ ]:
import numpy as np
from numba import njit
import os
import matplotlib.pyplot as plt
import gc

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3D

from matplotlib.animation import FuncAnimation, PillowWriter
from multiprocessing import Pool, cpu_count
import concurrent.futures as cf
from typing import Callable

In [ ]:

@njit(cache=True, fastmath=True)
def lorenz(xyz: np.ndarray, σ=10.0, ρ=28.0, β=8.0/3.0) -> np.ndarray:
    x, y, z = xyz
    dx = σ * (y - x)
    dy = x * (ρ - z) - y
    dz = x * y - β * z
    return np.array([dx, dy, dz])

@njit(cache=True, fastmath=True)
def euler(h: float, xyz: np.ndarray) -> np.ndarray:
    """Euler's method for solving ODEs. First-order accurate."""
    return xyz + h * lorenz(xyz)

@njit(cache=True, fastmath=True)
def heun(h: float, xyz: np.ndarray) -> np.ndarray:
    """Heun's method for solving ODEs. Second-order accurate."""
    k1 = lorenz(xyz)
    k2 = lorenz(xyz + 0.5 * h * k1)
    return xyz + h * k2

@njit(cache=True, fastmath=True)
def rk3(h: float, xyz: np.ndarray) -> np.ndarray:
    """Third-order Runge-Kutta method for solving ODEs. Third-order accurate."""
    k1 = lorenz(xyz)
    k2 = lorenz(xyz + 0.5 * h * k1)
    k3 = lorenz(xyz + 0.75 * h * k2)
    return xyz + h * (2.0 * k1 + 3.0 * k2 + 4.0 * k3) / 9.0

@njit(cache=True, fastmath=True)
def rk4(h: float, xyz: np.ndarray) -> np.ndarray:
    """Fourth-order Runge-Kutta method for solving ODEs. Fourth-order accurate."""
    k1 = lorenz(xyz)
    k2 = lorenz(xyz + 0.5 * h * k1)
    k3 = lorenz(xyz + 0.5 * h * k2)
    k4 = lorenz(xyz + h * k3)
    return xyz + h * (k1 + 2.0 * k2 + 2.0 * k3 + k4) / 6.0

In [ ]:
@njit(cache=True, fastmath=True)
def simulate_lorenz(
    steps: int=1000,
    h: float=0.02,
    initial_condition: np.ndarray =np.array([2.0, 1.0, 1.0]),
    method: Callable=rk4
) -> np.ndarray:
    """Simulate the Lorenz system using the fourth-order Runge-Kutta method."""
    xyz = np.zeros((steps, 3))
    xyz[0] = initial_condition

    for i in range(1, steps):
        xyz[i] = method(h, xyz[i-1])

    return xyz

In [ ]:
# def render_frame(args):
#     frame, points, method = args
#     xs, ys, zs = points[:, 0], points[:, 1], points[:, 2]
#     fig = plt.figure(figsize=(6, 6), dpi=140)
#     ax = fig.add_subplot(111, projection='3d')     
#     # Set the background color to black
#     ax.set_facecolor('black')
#     fig.set_facecolor('black')
#     # Remove the grid
#     ax.grid(False)
#     # Remove the axis ticks and labels
#     ax.set_xticks([])
#     ax.set_yticks([])
#     ax.set_zticks([])
#     ax.xaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
#     ax.yaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
#     ax.zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
#     ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
#     ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
#     ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    
#     # Create a gradient based on the length of xs up to frame
#     colors = plt.cm.inferno(np.linspace(0, 1, frame))
    
#     for i in range(1, frame):
#         ax.plot(xs[i-1:i+1], ys[i-1:i+1], zs[i-1:i+1], color=colors[i], linewidth=2.0)
#         ax.view_init(elev=30, azim=0.01 * i)

#     ax.set_xlim(min(xs), max(xs))
#     ax.set_ylim(min(ys), max(ys))
#     ax.set_zlim(min(zs), max(zs))
    
#     filename = f"frames_{method}/frame_{frame:04d}.png"
#     plt.savefig(filename, dpi=140, edgecolor='none', pad_inches=0, facecolor=fig.get_facecolor())
#     plt.close(fig)

#     del fig, ax, xs, ys, zs, colors
#     return filename


# def animate_lorenz(xyz: np.ndarray, method_name: str, frames_step: int=24):
#     frames = list(range(0, xyz.shape[0], frames_step))
#     print(f"Rendering {len(frames)} frames using {method_name}...")
#     with cf.ProcessPoolExecutor() as executor:
#         executor.map(render_frame, [(frame, xyz, method_name) for frame in frames])



In [ ]:
# def render_frame(args):
#     frame, points, method = args
#     xs, ys, zs = points[:, 0], points[:, 1], points[:, 2]
#     fig = plt.figure(figsize=(6, 6), dpi=140)
#     ax = fig.add_subplot(111, projection='3d')     
#     # Set the background color to black
#     ax.set_facecolor('black')
#     fig.set_facecolor('black')
#     # Remove the grid
#     ax.grid(False)
#     # Remove the axis ticks and labels
#     ax.set_xticks([])
#     ax.set_yticks([])
#     ax.set_zticks([])
#     ax.xaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
#     ax.yaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
#     ax.zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
#     ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
#     ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
#     ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    
#     # Create a gradient based on the length of xs up to frame
#     colors = plt.cm.inferno(np.linspace(0, 1, frame))
    
#     for i in range(1, frame):
#         ax.plot(xs[i-1:i+1], ys[i-1:i+1], zs[i-1:i+1], color=colors[i], linewidth=2.0)
#         ax.view_init(elev=30, azim=0.01 * i)

#     ax.set_xlim(min(xs), max(xs))
#     ax.set_ylim(min(ys), max(ys))
#     ax.set_zlim(min(zs), max(zs))
    
#     filename = f"frames_{method}/frame_{frame:04d}.png"
#     plt.savefig(filename, dpi=140, edgecolor='none', pad_inches=0, facecolor=fig.get_facecolor())
#     plt.close(fig)

#     del fig, ax, xs, ys, zs, colors
#     return filename


# def animate_lorenz(xyz: np.ndarray, method_name: str, frames_step: int=24):
#     frames = list(range(0, xyz.shape[0], frames_step))
#     print(f"Rendering {len(frames)} frames using {method_name}...")
#     with cf.ProcessPoolExecutor() as executor:



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from concurrent.futures import ProcessPoolExecutor

def render_frame(args):
    xyz, method_name, colors, frame_index = args

    xs, ys, zs = xyz[:, 0], xyz[:, 1], xyz[:, 2]

    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111, projection='3d')   
    ax.set_facecolor('black')
    fig.set_facecolor('black')
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.xaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.set_xlim(min(xs), max(xs))
    ax.set_ylim(min(ys), max(ys))
    ax.set_zlim(min(zs), max(zs))
    
    for j in range(1, frame_index + 1):
        ax.plot(xs[j-1:j+1], ys[j-1:j+1], zs[j-1:j+1], color=colors[j], linewidth=2.0)

    filename = f"frames_{method_name}/frame_{frame_index:04d}.png"
    plt.savefig(filename, dpi=140, edgecolor='none', pad_inches=0, facecolor=fig.get_facecolor())
    plt.close(fig)

    del fig, ax, xs, ys, zs
    gc.collect()

    return filename

def animate_lorenz(xyz: np.ndarray, method_name: str, frames_step: int=24):
    number_of_frames = xyz.shape[0]
    print(f"Rendering {number_of_frames} frames using {method_name}...")

    colors = plt.cm.inferno(np.linspace(0, 1, number_of_frames))
    frame_indices = list(range(1, number_of_frames, frames_step))

    with ProcessPoolExecutor() as executor:
        filenames = list(executor.map(render_frame, [(xyz, method_name, colors, i) for i in frame_indices]))

    print(f"Finished rendering {len(filenames)} frames.")



xyz_rk4 = simulate_lorenz(method=rk4, steps=10000, h=0.02)
animate_lorenz(xyz_rk4, 'rk4', frames_step=6)


In [ ]:
# xyz_rk4 = simulate_lorenz(method=rk4, steps=1000, h=0.02)

# # animate_lorenz(xyz_rk4, 'rk4', frames_step=12)

# # os.system("ffmpeg -framerate 24 -pattern_type glob -i 'frames/*.png' \
# #         -c:a copy -shortest -c:v libx264 -pix_fmt yuv420p out10fps.mp4")